# 3 Implémentation

In [1]:
from random import *

class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """

    def __init__(self, height, width):
        """
        Constructeur d'un labyrinthe de height cellules de haut
        et de width cellules de large
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height = height
        self.width = width
        self.neighbors = {(i, j): set() for i in range(height) for j in range(width)}

    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors) + "\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width - 1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width - 1):
            txt += "   ┃" if (0, j + 1) not in self.neighbors[(0, j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height - 1):
            txt += "┣"
            for j in range(self.width - 1):
                txt += "━━━╋" if (i + 1, j) not in self.neighbors[(i, j)] else "   ╋"
            txt += "━━━┫\n" if (i + 1, self.width - 1) not in self.neighbors[(i, self.width - 1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i + 1, j + 1) not in self.neighbors[(i + 1, j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width - 1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt

    def add_wall(self, c1, c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
               0 <= c1[1] < self.width and \
               0 <= c2[0] < self.height and \
               0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:  # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2)  # on le retire
        if c1 in self.neighbors[c2]:  # Si c1 est dans les voisines de c2
            self.neighbors[c2].remove(c1)  # on le retire
            
    def get_cells(self):
        """
        Méthode d'instance permettant d'obtenir les cellules de la grille du labyrinthe
        
        Retour : Liste de tuples représentant les cellules du labyrinthe
        """
        cells = []
        for i in range(self.height):
            for j in range(self.width):
                cells += [(i, j)]
        return cells
    
    def remove_wall(self, c1, c2):
        """
        Méthode d'instance permettant de retirer un mur
        
        Retour : Rien
        """
        if c1 not in self.neighbors[c2]:
            self.neighbors[c2].add(c1)
        if c2 not in self.neighbors[c1]:
            self.neighbors[c1].add(c2)
            
    def get_walls(self):
        """
        Méthode d'instance permettant d'obtenir la liste des murs du labyrinthe
        
        Retour : Liste de listes de deux tuples représentant les murs
        """
        walls = []
        for i in range(self.height):
            for j in range(self.width):
                if j + 1 < self.width and (i, j + 1) not in self.neighbors[(i, j)]:
                    walls += [[(i, j), (i, j + 1)]]
                if i + 1 < self.height and (i + 1, j) not in self.neighbors[(i, j)]:
                    walls += [[(i, j), (i + 1, j)]]
        return walls
    
    def fill(self):
        """
        Méthode d'instance permettant de remplir le labyrinthe des murs manquant
        
        Retour : Rien
        """
        walls = self.get_walls()
        for i in range(self.height):
            for j in range(self.width):
                if i + 1 < self.height and [(i, j), (i+1, j)] not in walls:
                    self.add_wall((i, j), (i+1, j))
                if j + 1 < self.width and (i, j+1) not in walls:
                    self.add_wall((i, j), (i, j+1))
    
    def empty(self) -> None:
        """
        Méthode d'instance permettant de retirer tous les murs restants du labyrinthe
        
        Retour : Rien
        """
        for cell in self.neighbors:
            if cell[0] < self.height - 1:
                self.remove_wall(cell, (cell[0] + 1, cell[1]))
            if cell[1] < self.width - 1:
                self.remove_wall(cell, (cell[0], cell[1] + 1))
    
    def get_contiguous_cells(self, c):
        """
        Méthode d'instance permettant de retourner les cellules contiguës à la cellule donnée en paramètre
        
        Retour : Liste de tuples représentant les cellules contiguës à la cellule c"""
        contiguous_cells = []
        if c[0] - 1 >= 0:
            contiguous_cells += [(c[0] - 1, c[1])]
        if c[0] + 1 < self.height:
            contiguous_cells += [(c[0] + 1, c[1])]
        if c[1] - 1 >= 0:
            contiguous_cells += [(c[0], c[1] - 1)]
        if c[1] + 1 < self.width:
            contiguous_cells += [(c[0], c[1] + 1)]
        return contiguous_cells
    
    def get_reachable_cells(self, c):
        """
        Méthode d'instance permettant de retourner les cellules accessible à partir de la cellule donnée en paramètre
        
        Retour : Liste de tuples représentant les cellules atteignables à partir de la cellule c"""
        reachable_cells = []
        for cell in self.get_contiguous_cells(c):
            if cell in self.neighbors[c]:
                reachable_cells += [cell]
        return reachable_cells
    
    
    @classmethod
    def gen_btree(cls, h, w):
        """
        Méthode de classe permettant de construire un labyrinthe avec l'algorithme de construction par arbre binaire
        
        Retour : Maze correspondant à un labbyrinthe
        """
        lab = Maze(h, w)
        for i in range(lab.height):
            for j in range(lab.width):
                walls = lab.get_walls()
                if (i + 1 < lab.height and [(i, j), (i+1, j)] in walls) and (j + 1 < lab.width and [(i, j), (i, j+1)] in walls):
                    choice = randint(0, 1)
                    if choice == 0:
                        lab.remove_wall((i, j), (i + 1, j))
                    else:
                        lab.remove_wall((i, j), (i, j + 1))
                elif i + 1 < lab.height and [(i, j), (i+1, j)] in walls and [(i, j), (i, j+1)] not in walls:
                    lab.remove_wall((i, j), (i + 1, j))
                elif j + 1 < lab.width and [(i, j), (i, j+1)] in walls and [(i, j), (i+1, j)] not in walls:
                        lab.remove_wall((i, j), (i, j + 1))
        return lab
    
    @classmethod
    def gen_sidewinder(cls, h, w):
        """
        Méthode de classe permettant de construire un labyrinthe avec l'algorithme de Sidewinder
        
        Retour : Maze représentant un labyrinthe
        """
        lab = Maze(h, w)
        for i in range(0, lab.height-1):
            sequence = []
            for j in range(0, lab.width-1):
                sequence += [(i, j)]
                choice = randint(0, 1)
                if choice == 0:
                    lab.remove_wall((i, j), (i, j+1))
                else :
                    choice_2 = randint(0, len(sequence)-1)
                    lab.remove_wall(sequence[choice_2],(sequence[choice_2][0]+1, sequence[choice_2][1]))
                    sequence = []
            sequence += [(i, lab.width-1)]
            choice = randint(0, len(sequence)-1)
            lab.remove_wall(sequence[choice],(sequence[choice][0]+1, sequence[choice][1]))
        for j in range(lab.width-1):
            lab.remove_wall((lab.height-1, j), (lab.height-1, j+1))
        return lab
    
    
    @classmethod
    def gen_fusion(cls, h, w):
        """
        Méthode de classe permettant de construire un labyrinthe avec l'algorithme par fusion de chemins
        
        Retour : Maze représentant un labyrinthe
        """
        lab = Maze(h, w)
        lab.fill()
        cells = lab.get_cells()
        cells_labels = {}
        for i in range(len(cells)):
            cells_labels[cells[i]] = i
        walls = lab.get_walls()
        shuffle(walls)
        for wall in walls:
            if cells_labels[wall[0]] != cells_labels[wall[1]]:
                lab.remove_wall(wall[0], wall[1])
                tmp = cells_labels[wall[1]]
                cells_labels[wall[1]] = cells_labels[wall[0]]
                for cell in cells_labels.keys():
                    if cells_labels[cell] == tmp :
                        cells_labels[cell] = cells_labels[wall[0]]
        return lab
    
    @classmethod
    def gen_exploration(cls, h, w):
        """
        Méthode de classe permettant de constuire un labyrinthe avec l'algorithme de génération par exploration
        
        Retour : Maze représentant un labyrinthe
        """
        lab = Maze(h,w)
        cells = lab.get_cells()
        shuffle(cells)
        random_cell = cells[0]
        visited = [random_cell]
        pile = [random_cell]
        while pile != [] :
            top_cell = pile.pop()
            visite = True
            i = 0
            while visite and i < len(lab.get_contiguous_cells(top_cell)):
                if lab.get_contiguous_cells(top_cell)[i] not in visited:
                    visite = False
                i += 1
            if not(visite):
                pile += [top_cell]
                contigues = lab.get_contiguous_cells(top_cell)
                contigues_non_visites = []
                for contigue in contigues:
                    if contigue not in visited:
                        contigues_non_visites += [contigue]
                shuffle(contigues_non_visites)
                lab.remove_wall(top_cell, contigues_non_visites[0])
                visited += [contigues_non_visites[0]]
                pile += [contigues_non_visites[0]]
        return lab
    
#    @classmethod
#    def gen_wilson(cls, h, w)
#    lab = Maze(h, w)
#    cells = lab.get_cells()
#    shuffle(cells)
#    marked_cells = cells[0]
#    while len(marked_cells) != len(cells):
#        marked = True
#        chosen_cell = ()
#        i = 0
#        while marked and i < len(cells):
#            if cells[i] not in marked_cells:
#                chosen_cell = cells[i]
#                marked = False
#            i += 1

    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #    self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt
    
    def solve_dfs(self, start, stop):
        """
        Méthode d'instance permettant de nous indiquer la solution pour résoudre le labyrinthe en partant de la cellule start
        en allant jusqu'à la cellule stop. Cet algorithme fonctionne à l'aide d'un parcours en profondeur.
        
        Retour : Liste de tuples représentant les coordonnées par lesquelles passer pour aller de la cellule 
        start à la cellule end
        """
        pile = [start]
        cells = self.get_cells()
        marked_cells = [start]
        pred = {}
        pred[start] = start
        trouve = False
        while not(trouve):
            c = pile.pop()
            if c == stop:
                trouve = True
            else:
                for cell in self.neighbors[c]:
                    if cell not in marked_cells:
                        marked_cells += [cell]
                        pile += [cell]
                        pred[cell] = c
        c = stop
        path = []
        while c != start:
            path += [c]
            c = pred[c]
        path += [start]
        return path
    
    def solve_bfs(self, start, stop):
        """
        Méthode d'instance permettant de nous indiquer la solution pour résoudre le labyrinthe en partant de la cellule start
        en allant jusqu'à la cellule stop. Cet algorithme fonctionne à l'aide d'un parcours en largeur.
        
        Retour : Liste de tuples représentant les coordonnées par lesquelles passer pour aller de la cellule 
        start à la cellule end
        """
        file = [start]
        cells = self.get_cells()
        marked_cells = [start]
        pred = {}
        pred[start] = start
        trouve = False
        while not(trouve) and len(marked_cells) != len(cells):
            c = file[0]
            del(file[0])
            if c == stop:
                trouve = True
            else:
                for cell in self.neighbors[c]:
                    if cell not in marked_cells:
                        marked_cells += [cell]
                        file += [cell]
                        pred[cell] = c
        c = stop
        path = []
        while c != start:
            path += [c]
            c = pred[c]
        path += [start]
        return path
    
    #def solve_rhr(self, start, stop):
    
    def distance_geo(self, c1, c2):
        """
        Méthode d'instance qui calcule la distance « géodésique » entre la cellule c1 et la cellule c2
        
        Retour : Entier représentant la distance à parcourir pour aller de la cellule c1 à c2
        """
        return len(self.solve_bfs(c1, c2))-1
    
    def distance_man(self, c1, c2):
        """
        Méthode d'instance qui calcule la distance de Manhattan, sur la grille, entre la cellule c1 et la cellule c2
        
        Retour : Entier représentant la distance à parcours pour aller de la cellule c1 à c2 en ignorant les murs
        """
        return abs(c1[0]-c2[0]) + abs(c1[1]-c2[1])
    
    #def worst_path_len(self):
    
    def dead_end_number(self):
        dead_end = 0
        cells = self.get_cells()
        for cell in cells:
            if len(self.neighbors[cell]) == 1:
                dead_end += 1
        return dead_end
        
       
    

# Partie Tests

## Tests partie 4

In [2]:
#Tests sur les fonctions get_cells() et get_walls()

lab = Maze(5, 6)
print(lab)
print(lab.get_cells())
print()
print(lab.get_walls())


┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]

[[(0, 0), (0, 1)], [(0, 0), (1, 0)], [(0, 1), (0, 2)], [(0, 1), (1, 1)], [(0, 2), (0, 3)], [(0, 2), (1, 2)], [(0, 3), (0, 4)], [(0, 3), (1, 3)], [(0, 4), (0, 5)], [(0, 4), (1, 4)], [(0, 5), (1, 5)], [(1, 0), (1, 1)], [(1, 0), (2, 0)], [(1, 1), (1, 2)], [(1, 1), (2, 1)], [(1, 2), (1, 3)], [(1, 2), (2, 2)], [(1, 3), (1, 4)], [(1, 3), (2, 3)], [(1, 4), (1, 5)], [(1, 4), (2, 4)], [(1, 5), (2, 5)], [(2, 0), (2, 1)], [(2, 0), (3, 0)], [(2, 1), (2, 2)], [(2, 1), (3, 1)], [(

In [3]:
#Tests sur les fonctions empty() et fill()

lab.empty()
print(lab)
lab.fill()
print(lab)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃                       ┃
┣   ╋   ╋   ╋   ╋   ╋   ┫
┃                       ┃
┣   ╋   ╋   ╋   ╋   ╋   ┫
┃                       ┃
┣   ╋   ╋   ╋   ╋   ╋   ┫
┃                       ┃
┣   ╋   ╋   ╋   ╋   ╋   ┫
┃                       ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛



In [4]:
#Tests sur les fonctions get_contiguous_cells() et get_reachable_cells()

cont = lab.get_contiguous_cells((4,5))
print(cont)
lab.remove_wall((1,1), (1,2))
print(lab)
reach = lab.get_reachable_cells((1,2))
print(reach)



[(3, 5), (4, 4)]
┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛

[(1, 1)]


## Tests parti 5

In [5]:
#Test sur la méthode de classe gen_btree()

lab2 = Maze.gen_btree(15, 15)
print(lab2)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃       ┃               ┃       ┃   ┃   ┃           ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ┫
┃       ┃           ┃           ┃           ┃   ┃       ┃   ┃
┣━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃   ┃   ┃   ┃       ┃           ┃   ┃       ┃           ┃   ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃   ┃   ┃   ┃       ┃       ┃               ┃           ┃   ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃       ┃           ┃           ┃               ┃       ┃   ┃
┣━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃   ┃   ┃   ┃   ┃   ┃                       ┃   ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃   ┃       ┃   ┃               ┃       ┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ┫
┃               ┃   ┃               ┃   ┃   ┃   ┃           ┃
┣━━━╋━━━

In [6]:
#Test sur la méthode de classe gen_sidewinder()

lab3 = Maze.gen_sidewinder(15, 15)
print(lab3)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃               ┃                       ┃
┣   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃       ┃   ┃   ┃                       ┃   ┃
┣   ╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃   ┃       ┃               ┃   ┃   ┃   ┃       ┃   ┃       ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃       ┃       ┃           ┃   ┃           ┃       ┃       ┃
┣   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ┫
┃           ┃   ┃               ┃   ┃       ┃       ┃   ┃   ┃
┣━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃               ┃       ┃       ┃   ┃   ┃   ┃       ┃       ┃
┣━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━┫
┃                               ┃       ┃           ┃       ┃
┣━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ┫
┃   ┃               ┃   ┃   ┃   ┃       ┃   ┃       ┃       ┃
┣   ╋━━━

In [7]:
#Test sur la méthode de classe gen_fusion()

lab4 = Maze.gen_fusion(15, 15)
print(lab4)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃               ┃                       ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━┫
┃   ┃                       ┃       ┃   ┃   ┃           ┃   ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ┫
┃               ┃   ┃       ┃           ┃       ┃           ┃
┣   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ┫
┃   ┃               ┃       ┃       ┃           ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━┫
┃       ┃               ┃       ┃   ┃   ┃   ┃       ┃       ┃
┣━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃   ┃                   ┃   ┃   ┃       ┃       ┃           ┃
┣   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ┫
┃       ┃           ┃   ┃           ┃           ┃   ┃   ┃   ┃
┣━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋   ┫
┃       ┃   ┃       ┃           ┃   ┃   ┃   ┃   ┃           ┃
┣   ╋━━━

In [8]:
#Tests sur la méthode de classe gen_exploration()

lab5 = Maze.gen_exploration(15, 15)
print(lab5)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃                           ┃                   ┃           ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃       ┃   ┃       ┃       ┃       ┃   ┃   ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋   ┫
┃   ┃   ┃           ┃   ┃       ┃       ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃       ┃   ┃           ┃   ┃       ┃           ┃   ┃   ┃   ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃       ┃       ┃           ┃   ┃   ┃           ┃       ┃   ┃
┣━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ┫
┃       ┃   ┃       ┃       ┃   ┃                   ┃   ┃   ┃
┣   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃   ┃       ┃   ┃           ┃       ┃   ┃       ┃           ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━┫
┃       ┃   ┃           ┃       ┃   ┃       ┃               ┃
┣━━━╋   

## Tests partie 6

In [9]:
#Tests sur la méthode d'instance solve_dfs()

path = lab5.solve_dfs((0,0), (14, 14))
print(path)
print()
solution = {c : '*' for c in path}
solution[(0,0)] = 'D'
solution[(14, 14)] = 'A'
print(lab5.overlay(solution))

[(14, 14), (14, 13), (14, 12), (13, 12), (12, 12), (12, 13), (13, 13), (13, 14), (12, 14), (11, 14), (10, 14), (9, 14), (8, 14), (7, 14), (7, 13), (7, 12), (7, 11), (6, 11), (6, 10), (7, 10), (7, 9), (8, 9), (9, 9), (9, 8), (8, 8), (7, 8), (6, 8), (6, 7), (5, 7), (4, 7), (3, 7), (3, 8), (2, 8), (2, 9), (1, 9), (1, 8), (0, 8), (0, 7), (1, 7), (1, 6), (2, 6), (3, 6), (4, 6), (4, 5), (4, 4), (5, 4), (5, 3), (6, 3), (7, 3), (7, 4), (7, 5), (8, 5), (9, 5), (9, 6), (8, 6), (7, 6), (7, 7), (8, 7), (9, 7), (10, 7), (10, 6), (10, 5), (11, 5), (11, 4), (12, 4), (12, 3), (11, 3), (11, 2), (11, 1), (10, 1), (10, 2), (9, 2), (9, 1), (9, 0), (8, 0), (8, 1), (7, 1), (7, 0), (6, 0), (5, 0), (5, 1), (4, 1), (4, 0), (3, 0), (3, 1), (2, 1), (1, 1), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (3, 5), (2, 5), (1, 5), (1, 4), (0, 4), (0, 3), (0, 2), (0, 1), (0, 0)]

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *   *   *   *         ┃ *   *             ┃           ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━

In [10]:
#Tests sur la méthode d'instance solve_bfs()

path = lab5.solve_bfs((0,0), (14, 14))
print(path)
print()
solution = {c : '*' for c in path}
solution[(0,0)] = 'D'
solution[(14, 14)] = 'A'
print(lab5.overlay(solution))

[(14, 14), (14, 13), (14, 12), (13, 12), (12, 12), (12, 13), (13, 13), (13, 14), (12, 14), (11, 14), (10, 14), (9, 14), (8, 14), (7, 14), (7, 13), (7, 12), (7, 11), (6, 11), (6, 10), (7, 10), (7, 9), (8, 9), (9, 9), (9, 8), (8, 8), (7, 8), (6, 8), (6, 7), (5, 7), (4, 7), (3, 7), (3, 8), (2, 8), (2, 9), (1, 9), (1, 8), (0, 8), (0, 7), (1, 7), (1, 6), (2, 6), (3, 6), (4, 6), (4, 5), (4, 4), (5, 4), (5, 3), (6, 3), (7, 3), (7, 4), (7, 5), (8, 5), (9, 5), (9, 6), (8, 6), (7, 6), (7, 7), (8, 7), (9, 7), (10, 7), (10, 6), (10, 5), (11, 5), (11, 4), (12, 4), (12, 3), (11, 3), (11, 2), (11, 1), (10, 1), (10, 2), (9, 2), (9, 1), (9, 0), (8, 0), (8, 1), (7, 1), (7, 0), (6, 0), (5, 0), (5, 1), (4, 1), (4, 0), (3, 0), (3, 1), (2, 1), (1, 1), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (3, 5), (2, 5), (1, 5), (1, 4), (0, 4), (0, 3), (0, 2), (0, 1), (0, 0)]

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *   *   *   *         ┃ *   *             ┃           ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━

## Tests partie 7

In [11]:
#Test de la méthode d'instance distance_geo()

print(lab5.distance_geo((0, 0), (14, 14)))

100


In [12]:
#Test de la méthode d'instance distance_man()

print(lab5.distance_man((0, 0), (14, 14)))

28


In [13]:
#Test de la méthode d'instance dead_end_number()

lab6 = Maze.gen_exploration(5, 6)
print(lab6)
print(lab6.dead_end_number())

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃                       ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃   ┃               ┃   ┃
┣   ╋   ╋━━━╋━━━╋   ╋   ┫
┃   ┃       ┃   ┃   ┃   ┃
┣   ╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃       ┃   ┃   ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ┫
┃                   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛

3


## Parie 8

Pour compliquer ce labyrinthe, on pourrait implémenter un système qui permettrait de modifier quelques murs chaque fois qu'un temps en particulier soit écoulé, je dirais 5 à 10 secondes, peut-être même de manière aléatoire pour ajouter un peu plus de suspense dans la complétion du labyrinthe. De ce fait, on pourrait déjà avoir un peu plus de difficulté à résoudre le labyrinthe.  

Une autre manière de compléxifier le labyrinthe serait d'y implémenter des zones dites << aveugles >>, couverte de noir, où il serait impossible de voir les différents chemins à l'intérieur de ces zones.  

Une dernière manière qui me vient à l'esprit pour compliquer la résolution visuelle serait de créer des murs invisibles et des murs qui peuvent être traversés pour ajouter de la difficulté à la résolution du labyrinthe. Le taux de génération de ses murs invisibles et traversables pourrait être contrôlé pour éviter que ça soit trop compliqué à résoudre. Le but serait surtout de rajouter quelques petits murs invisibles et traversables pour créer des impasses mais aussi créer des issues cachées.